<a href="https://colab.research.google.com/github/mangalm96/HP-TopicModeling/blob/main/INFO_202_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import and Installation

In [3]:
# import libraries
import os
import re
!pip install path
!pip install -q pandas
!pip install -q gensim
!pip install -q pynndescent>=0.4
!pip install -q umap-learn
!pip install -q hdbscan
!pip install -q wordcloud
!pip install -q tensorflow
!pip install -q tensorflow_hub
!pip install -q tensorflow_text
!pip install -q torch
!pip install -q sentence_transformers
!pip install -q hnswlib

     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 6.4 MB 28.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.9 MB 22.7 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
     |████████████████████████████████| 3.4 MB 24.0 MB/s 
     |████████████████████████████████| 3.3 MB 23.4 MB/s 
     |████████████████████████████████| 1.2 MB 40.0 MB/s 
     |████████████████████████████████| 61 kB 472 kB/s 
     |████████████████████████████████| 895 kB 69.9 MB/s 
     |████████████████████████████████| 596 kB 39.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# top2vec installations
!pip install -q top2vec[sentence_transformers]
!pip install -q numpy==1.20.0

In [8]:
# import packages for top2vec
# if doesn't run in first go, run again
import numpy as np
import pandas as pd
from copy import deepcopy
from top2vec import Top2Vec
import time

#### Read dataset and create chapters

In [9]:
## read HP text

folderpath = '/content/drive/My Drive/HP_Books'
entries = os.listdir(folderpath)
filepaths  = [os.path.join(folderpath, name) for name in entries]

# initialise dict which will contain all HP books
hp_book ={}

for book,path in zip(entries,filepaths):
    list_lines=[]
    with open(path) as f:

        # parse text before .txt to get book name
        book_name = book.split(".")[0]
        
        # remove "\n" and blanks
        for line in f:
            # remove \n
            cleaned_line = line.rstrip('\n')
            # remove blank lines
            if cleaned_line != "":
                # remove \xad instances 
                cleaned_line = cleaned_line.replace('\xad', '')
                 # remove \x0c instances 
                cleaned_line = cleaned_line.replace('\x0c', '')
                # append to a list of lines
                list_lines.append(cleaned_line)
         
        # add to dict as per book name
        hp_book[book_name] = list_lines



In [10]:
# length of each book -- this length might not be 100% correct. Stay tuned for further parsing
for name,bk in hp_book.items():
  print(name,":",len(bk))

1-sorcerers-stone : 3032
3-the-prisoner-of-azkaban : 10538
4-the-goblet-of-fire : 6809
5-order-of-the-phoenix : 18954
6-the-half-blood-prince : 5864
7-deathly-hallows : 6588
2-chamber-of-secrets : 8718


In [11]:
hp_book['4-the-goblet-of-fire'][3]

'The villagers of Little Hangleron still called it "the Riddle House," even though it had been many years since the Riddle family had lived there. It stood on a hill overlooking the village, some of its windows boarded, tiles missing from its roof, and ivy spreading unchecked over its face. Once a finelooking manor, and easily the largest and grandest building for miles around, the Riddle House was now damp, derelict, and unoccupied.'

#### Create chapters on all books

In [12]:
# pick all books together 


all_books = []
for bk_name in hp_book.keys():
  entire_text =[]
  entire_text=' '.join(hp_book[bk_name])
  all_books.append(entire_text)

complete_text = " ".join(all_books)

# divide into chapters
rgr = r'CHAPTER|Chapter\s[A-Z]{3,}'
chapter_wise_all = re.split(rgr,complete_text)
title = chapter_wise_all.pop(0)

# split on last two books
rgr_new = r'Chapter\s\d+'
remain_text = re.split(rgr_new,chapter_wise_all[-1])

# split on last book
rgr_new_2 = r'Chapter\s[A-za-z]{3,}'
remain_text_full = re.split(rgr_new_2,remain_text[-1])

# split on 2nd book
rgr_new_hp2 = r'\d{1,2}'
remain_text_fullfull = re.split(rgr_new_hp2,remain_text_full[-1])

# merge all three lists together 
full_text = chapter_wise_all[:-1] + remain_text[:-1] + remain_text_full[:-1] + remain_text_fullfull




# # join all elements in list together 
# entire_text=' '.join(hp_book['3-the-prisoner-of-azkaban'])



In [13]:
len(full_text)

216

In [14]:
len(chapter_wise_all)

114

In [15]:
len(remain_text)

31

In [16]:
len(remain_text_full)

37

In [17]:
len(remain_text_fullfull)

37

In [18]:
remain_text_fullfull[12]

'pm “We can actually meet him!” Hermione squealed. “I mean, he’s written almost the whole booklist!” The crowd seemed to be made up mostly of witches around Mrs Weasley’s age. A harassed-looking wizard stood at the door, saying, “Calmly, please ladies … don’t push, there … mind the books, now …” Harry, Ron and Hermione squeezed inside. A long queue wound right to the back of the shop, where Gilderoy Lockhart was signing his books. They each grabbed a copy of Break with a Banshee, and sneaked up the line to where the rest of the Weasleys were standing with Mr and Mrs Granger. “Oh, there you are, good,” said Mrs Weasley. She sounded breathless and kept patting her hair. “We’ll be able to see him in a minute …” Gilderoy Lockhart came slowly into view, seated at a table surrounded by large pictures of his own face, all winking and flashing dazzlingly white teeth at the crowd. The real Lockhart was wearing robes of forget-me-not blue which exactly matched his eyes; his pointed wizard’s hat 

In [ ]:
# find number of characters in each
ct=1
for ch in full_text:
  # print(ct)a
  print(len(ch))
  # ct+=1

In [ ]:
# find number of characters in each
ct=1
for ch in full_text:
  # print(ct)a
  print(len(ch))
  # ct+=1

In [126]:
## each chapter will acts as a document for my Top2Vec algorithm 
# run top2vec model 
start_time = time.time()
# model_full = Top2Vec(documents=full_text,speed='learn',workers=8)  # training using Doc2Vec
# model_full_2 = Top2Vec(documents=full_text,speed='learn')  # training using Doc2Vec
model_full_3 = Top2Vec(documents=full_text,speed='deep-learn',workers=10)  # training using Doc2Vec


print('Total time taken (mins): ', int((time.time()-start_time)/60))

2021-12-15 13:00:24,951 - top2vec - INFO - Pre-processing documents for training
2021-12-15 13:00:28,037 - top2vec - INFO - Creating joint document/word embedding
2021-12-15 13:23:24,728 - top2vec - INFO - Creating lower dimension embedding of documents
2021-12-15 13:23:28,332 - top2vec - INFO - Finding dense areas of documents
2021-12-15 13:23:28,342 - top2vec - INFO - Finding topics


Total time taken (mins):  23


In [127]:
model_full_3.get_num_topics()
topic_words, word_scores, topic_nums = model_full_3.get_topics(model_full_3.get_num_topics())
topic_words
## topic 2 def related to death, pain ,fear, volderort, death eater, kill
## topic 1 could be hogwarts related -- classes, dorm, houses, detention, prof, quidditch
## topic 3 could be related to family and friends with whom interaction is outside hogwarts -- dursleys, weasleys, godfather, tonks, burrow, sirius


array([['lesson', 'common', 'umbridge', 'class', 'professor', 'ron',
        'hermione', 'just', 'lavender', 'homework', 'lessons',
        'classroom', 'about', 'during', 'dormitory', 'really', 'looking',
        'some', 'gryffindor', 'malfoy', 'teacher', 'practice', 'goyle',
        'slytherins', 'potions', 'detention', 'students', 'gryffindors',
        'quidditch', 'parvati', 'to', 'team', 'doing', 'yeah', 'look',
        'up', 'mcgonagall', 'when', 'though', 'crabbe', 'cho', 'library',
        'towards', 'alicia', 'said', 'angelina', 'pitch', 'lunch',
        'corridors', 'match'],
       ['voldemort', 'death', 'lord', 'bellatrix', 'pain', 'eaters',
        'body', 'darkness', 'fear', 'horcrux', 'felt', 'wand', 'upon',
        'knew', 'griphook', 'beneath', 'wormtail', 'ariana', 'light',
        'could', 'kill', 'elder', 'dead', 'him', 'hallows', 'eater',
        'he', 'beyond', 'eyes', 'man', 'but', 'raised', 'the', 'scar',
        'not', 'narcissa', 'killed', 'then', 'as', 'grou

In [206]:
with open('/content/drive/My Drive/model_full_3.pickle', 'wb') as handle:
  pickle.dump(model_full_3, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [179]:
### similarity exercises

# Get Topic Sizes
topic_sizes, topic_nums = model_full_3.get_topic_sizes()
topic_sizes #array([81, 61, 38]) total 180


array([0, 1, 2])

In [186]:
# We are going to search for topics most similar to medicine.


topic_words, word_scores, topic_scores, topic_nums = model_full_3.search_topics(keywords=["family"], num_topics=3)
topic_scores

# with love --array([0.15463559, 0.14641942, 0.10825591])
# with death -- array([0.4401925 , 0.09775307, 0.08141368])


array([0.29656357, 0.17520449, 0.05015621])

In [203]:
documents, document_scores, document_ids = model_full_3.search_documents_by_keywords(keywords=["death"], num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 110, Score: 0.39346981048583984
-----------
 THIRTY-FIVE Beyond the Veil Black shapes were emerging out of thin air all around them, blocking their way left and right; eyes glinted through slits in hoods, a dozen lit wand tips were pointing directly at their hearts; Ginny gave a gasp of horror. "To me, Potter," repeated the drawling voice of Lucius Malfoy as he held out his hand, palm up. Harrys insides plummeted sickeningly. They were trapped, and outnumbered two to one. "To me," said Malfoy yet again. "Where’s Sirius?" Harry said. Several of the Death Eaters laughed; a harsh female voice from the midst of the shadowy figures to Harry’s left said triumphantly, "The Dark Lord always knows!" "Always," echoed Malfoy softly. "Now, give me the prophecy Potter." "I want to know where Sirius is!" "I want to know where Sirius is!" mimicked the woman to his left. She and her fellow Death Eaters had closed in so that they were mere feet away from Harry and the others, the light from t

In [202]:
documents, document_scores, document_ids = model_full_3.search_documents_by_keywords(keywords=["love"], num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 128, Score: 0.22720858454704285
-----------
: The Unbreakable Vow Snow was swirling against the icy windows once more; Christmas was approaching fast. Hagrid had already singlehandedly delivered the usual twelve C hristmas trees to the Great Hall; garlands of holly and tinsel had been twisted around the banisters of the stairs; everlasting candles glowed from inside the helmets of suits of armor and great bunches of mistletoe had been hung at intervals along the corridors. Large groups of girls tended to converge underneath the mistletoe bunches every time Harry went past, which caused blockages in the corridors; fortunat e ly, however, Harry's frequent nighttime wanderings had given him an unusually good knowledge of the castle's secret passageways, so that he was often, without too much difficulty, to naviga t e mistletoe-free routes between classes. Ron, who might once have found the necessity of these detours excuse for jealousy rather than hilarity, simply roared with la

In [191]:
documents, document_scores, document_ids = model_full_3.search_documents_by_keywords(keywords=["brave"], num_docs=3)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 73, Score: 0.18862423300743103
-----------
 THIRTYFIVE  VERITASERUM Harry felt himself slam flat into the ground; his face was pressed into grass; the smell of it filled his nostrils. He had closed his eyes while the Portkey transported him, and he kept them closed now. He did not move. All the breath seemed to have been knocked out of him; his head was swimming so badly he felt as though the ground beneath him were swaying like the deck of a ship. To hold himself steady, he tightened his hold on the two things he was still clutching: the smooth, cold handle of the Triwizard Cup and Cedric's body. He felt as though he would slide away into the blackness gathering at the edges of his brain if he let go of either of them. Shock and exhaustion kept him on the ground, breathing in the smell of the grass, waiting . . . waiting for someone to do something . . . something to happen . . . and all the while, his scar burned dully on his forehead. . . . A torrent of sound deafened and 

In [201]:
documents, document_scores, document_ids = model_full_3.search_documents_by_keywords(keywords=["friends"], num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 16, Score: 0.19759389758110046
-----------
 SEVENTEEN  	 THE MAN WITH TWO FACES It was Quirrell.  	 "You!" gasped Harry.  	 Quirrell smiled. His face wasn't twitching at all.  	 "Me," he said calmly. "I wondered whether I'd be meeting you here, Potter."  	 "But I thought -- Snape --"  	 "Severus?" Quirrell laughed, and it wasn't his usual quivering treble, either, but cold and sharp. "Yes, Severus does seem the type, doesn't he? So useful to have him swooping around like an overgrown bat. Next to him, who would suspect p-p-poor, st-stuttering P-Professor Quirrell?"  	 Harry couldn't take it in. This couldn't be true, it couldn't.  	 "But Snape tried to kill me!"  	 "No, no, no. I tried to kill you. Your friend Miss Granger accidentally knocked me over as she rushed to set fire to Snape at that Quidditch match. She broke my eye contact with you. Another few seconds and I'd have got you off that broom. I'd have managed it before then if Snape hadn't been muttering a countercurs

#### Model 3 with HP2 not separated and chapter used as docs.

In [49]:
# pick all books together 


all_books = []
for bk_name in hp_book.keys():
  entire_text =[]
  entire_text=' '.join(hp_book[bk_name])
  all_books.append(entire_text)

complete_text = " ".join(all_books)

# divide into chapters
rgr = r'CHAPTER|Chapter\s[A-Z]{3,}'
chapter_wise_all = re.split(rgr,complete_text)
title = chapter_wise_all.pop(0)

# split on last two books
rgr_new = r'Chapter\s\d+'
remain_text = re.split(rgr_new,chapter_wise_all[-1])

# split on last book
rgr_new_2 = r'Chapter\s[A-za-z]{3,}'
remain_text_full = re.split(rgr_new_2,remain_text[-1])

# split on 2nd book
# rgr_new_hp2 = r'\d{1,2}'
# remain_text_fullfull = re.split(rgr_new_hp2,remain_text_full[-1])

# merge all three lists together 
# full_text_4_3 = chapter_wise_all[:-1] + remain_text[:-1] + remain_text_full[:-1] + remain_text_fullfull
full_text_4_3 = chapter_wise_all[:-1] + remain_text[:-1] + remain_text_full


# # join all elements in list together 
# entire_text=' '.join(hp_book['3-the-prisoner-of-azkaban'])



In [52]:
for ch in full_text_4_3:
  print(len(ch))

25946
19189
21573
20431
37686
35064
25322
17253
28278
24225
19231
30800
18333
19699
29414
37078
31629
20854
21915
24886
29368
32355
35760
24883
29564
30473
41121
29504
27863
24585
31590
32598
22450
23877
13065
29314
11338
42216
27776
23005
16008
17813
17413
21575
13697
30381
34082
41409
18485
19052
32111
23165
27952
30673
35612
23831
38229
36283
40137
32119
25284
47018
36382
31838
45737
40176
42841
24902
36716
44972
11425
23042
16698
33430
35415
28015
33138
35020
29880
32486
31197
39024
26736
24636
46174
34039
34779
48972
50648
46189
40649
33928
38916
39669
38713
34470
42201
46486
42758
46332
46605
48968
43800
46345
43008
47253
46066
36013
22280
29418
44135
21875
45212
43430
29496
27552
27775
34921
34260
34356
38018
24691
35117
34607
29694
31373
31153
37527
34266
35577
36957
41740
36220
37841
32867
33826
33347
34099
29374
36573
27538
20669
32228
31051
18135
23650
19065
30432
33557
37584
37943
35882
23300
38089
32200
35556
34029
23943
40134
29776
30267
20030
37312
25818
27736
33561
4526

In [53]:
len(full_text_4_3)

180

In [82]:
#### model 4 with HP2 separated

## each chapter will acts as a document for my Top2Vec algorithm 
# run top2vec model 
start_time = time.time()
# model_full = Top2Vec(documents=full_text,speed='learn',workers=8)  # training using Doc2Vec
# model_full_2 = Top2Vec(documents=full_text,speed='learn')  # training using Doc2Vec
model_full_3 = Top2Vec(documents=full_text_4_3,speed='deep-learn',workers=12) # training using Doc2Vec
print('Total time taken (mins): ', int((time.time()-start_time)/60))


2021-12-15 22:54:09,397 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2021-12-15 22:54:14,525 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2021-12-15 23:16:56,423 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2021-12-15 23:16:59,917 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2021-12-15 23:16:59,936 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


Total time taken (mins):  22


In [92]:
model_full_3.get_num_topics()
topic_words, word_scores, topic_nums = model_full_3.get_topics(model_full_3.get_num_topics())
topic_words
## topic 2 def related to death, pain ,fear, volderort, death eater, kill
## topic 1 could be hogwarts related -- classes, dorm, houses, detention, prof, quidditch
## topic 3 could be related to family and friends with whom interaction is outside hogwarts -- dursleys, weasleys, godfather, tonks, burrow, sirius


array([['hermione', 'ron', 'common', 'umbridge', 'just', 'lesson',
        'class', 'lessons', 'professor', 'team', 'up', 'classroom',
        'yeah', 'potions', 'lavender', 'doing', 'corridors', 'really',
        'though', 'about', 'girls', 'gryffindor', 'feeling', 'all',
        'homework', 'quidditch', 'teacher', 'when', 'dormitory', 'harry',
        'snape', 'at', 'angelina', 'students', 'slytherins', 'them',
        'said', 'during', 'oh', 'or', 'neville', 'defense', 'detention',
        'to', 'off', 'on', 'she', 'first', 'malfoy', 'that'],
       ['voldemort', 'body', 'death', 'wand', 'could', 'fear',
        'bellatrix', 'darkness', 'elder', 'eaters', 'horcrux', 'pain',
        'kill', 'griphook', 'ariana', 'light', 'snake', 'lord', 'cold',
        'felt', 'lay', 'knew', 'he', 'there', 'upon', 'dead', 'sword',
        'beneath', 'eyes', 'narcissa', 'beyond', 'where', 'as',
        'wormtail', 'its', 'xenophilius', 'close', 'locket', 'place',
        'man', 'then', 'hallows', 'se

In [94]:
for w,s in zip(topic_words[0],word_scores[0]):
  print(w,s)

hermione 0.44395873
ron 0.44234654
common 0.42440963
umbridge 0.41901335
just 0.4041715
lesson 0.3984521
class 0.39492255
lessons 0.3860039
professor 0.38412344
team 0.38090414
up 0.3790562
classroom 0.3703401
yeah 0.36760208
potions 0.36407015
lavender 0.36139235
doing 0.35792628
corridors 0.3557569
really 0.35525155
though 0.3535346
about 0.35300875
girls 0.35224494
gryffindor 0.35087258
feeling 0.3500796
all 0.3500146
homework 0.3499324
quidditch 0.3493333
teacher 0.3493119
when 0.34706527
dormitory 0.3464824
harry 0.34133494
snape 0.34089717
at 0.3405743
angelina 0.3391167
students 0.33851716
slytherins 0.3361734
them 0.33604467
said 0.3355145
during 0.33518556
oh 0.33499497
or 0.33445907
neville 0.33357403
defense 0.33195117
detention 0.33030418
to 0.32963842
off 0.32888296
on 0.3287041
she 0.32726383
first 0.32418734
malfoy 0.3239525
that 0.32323462


In [ ]:
# make bar chart with selected words




In [85]:
documents, document_scores, document_ids = model_full_3.search_documents_by_keywords(keywords=["love"], num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 128, Score: 0.23047322034835815
-----------
: The Unbreakable Vow Snow was swirling against the icy windows once more; Christmas was approaching fast. Hagrid had already singlehandedly delivered the usual twelve C hristmas trees to the Great Hall; garlands of holly and tinsel had been twisted around the banisters of the stairs; everlasting candles glowed from inside the helmets of suits of armor and great bunches of mistletoe had been hung at intervals along the corridors. Large groups of girls tended to converge underneath the mistletoe bunches every time Harry went past, which caused blockages in the corridors; fortunat e ly, however, Harry's frequent nighttime wanderings had given him an unusually good knowledge of the castle's secret passageways, so that he was often, without too much difficulty, to naviga t e mistletoe-free routes between classes. Ron, who might once have found the necessity of these detours excuse for jealousy rather than hilarity, simply roared with la